In [1]:
import Base: iterate, exp, log, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero,max, isless
using BenchmarkTools

In [2]:
# The elements are `value`, the result of a computation; 
# `dfdy` a storage location where the derivative of some overall 
# computation (`f(value)`) with respect to the output of this element is stored;

# `parent` storing the "parent" locations that have been used to produce the result `value`;
# `backpropagation!` a function that propogates the derivative `dfdy` to the
# parents via `dfdx = dydx * dfdy` where `dydx` is the derivative of this piece
# of the computation, `dfdy` is the derivative of the overall function with
# respect to the output, and `dfdx` is the same for the next level up.
mutable struct Dual{T <: Number} <: Number
    value::T
    dfdy::T
    parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}
    backpropagation!
end
# Dual(n::Integer, d::Float64, parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}, bp!::Any)  where {T <: Number} = Dual(promote(n, d)..., parent, bp!)
# Dual(n::Float64, d::Integer, parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}, bp!::Any)  where {T <: Number} = Dual(promote(n, d)..., parent, bp!)


In [3]:
# x = Dual(2.0, 2, 1, (dfdy, parents) -> nothing)*  Dual(2, 2, 1, (dfdy, parents) -> nothing)
# promote_type(Dual{Real}, Dual{Number})

In [4]:
function convert(::Type{Dual{T}}, x::T) where T <: Number
    Dual(x, zero(T), nothing, (dfdy, parents) -> nothing)
end

# function convert(::Type{Dual{T}}, x::Dual{S}) where {T, S <: Number}
#     Dual(T(x.value), T(x.dfdy), x.parent, x.backpropagation!)
# end


function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
#     print("2asdsadsda", typeof(x), "\n")
    Dual(T(x), zero(T), nothing, (dfdy, parents) -> nothing)
end

function convert(::Type{Dual{T}}, x::Dual{T}) where T
    x
end


convert (generic function with 190 methods)

In [5]:
function zero(x::Dual{T}) where T
    Dual(zero(T), zero(T), nothing, (dfdy, parents) -> nothing)
end

zero (generic function with 23 methods)

In [6]:
# function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
#     Dual{promote_type(T,S)}
# end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end

function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end

function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S, T)}
end


promote_rule (generic function with 127 methods)

In [7]:
function push_parents!(queue::Array{Dual{T}, 1}, ::Nothing) where T
    # Do nothing
end
function push_parents!(queue::Array{Dual{T}, 1}, i::Int) where T
    # Do nothing
end
function push_parents!(queue::Array{Dual{T}, 1}, ls::Array{Dual{T}, 1}) where T
    append!(queue, ls)
end
function push_parents!(queue::Array{Dual{T}, 1}, l::Dual{T}) where T
    push!(queue, l)
end

push_parents! (generic function with 4 methods)

In [8]:
function backprop!(l::Dual{T}) where T
    # Apparently we need this construction because otherwise l gets copied when
    # put into the array.
    backprop!([l])
end
function backprop!(queue::Array{Dual{T},1}) where T
    while length(queue) > 0
        l = popfirst!(queue)
        l.backpropagation!(l.dfdy, l.parent)
        push_parents!(queue, l.parent)
    end
end

backprop! (generic function with 2 methods)

In [9]:
function collect_outputs(l::Dual{T}) where T
    queue = Dual{T}[l]

    outputs = Dual{T}[]

    while length(queue) > 0
        l = popfirst!(queue)
        if typeof(l.parent) <: Int
            push!(outputs, l)
        elseif typeof(l.parent) == Dual{T}
            push!(queue, l.parent)
        elseif typeof(l.parent) == Array{Dual{T}, 1}
            append!(queue, l.parent)
        else # Nothing
            # Do nothing
        end
    end

    outputs
end


collect_outputs (generic function with 1 method)

In [10]:
function derivativeСalculation(f)
    function dfdx(x::T) where T <: Number
        # Pass the function a backward infinitesimal whose backprop function
        # stores the backprop derivative in dfdx_store
#         print("x::T", "\n")

        x = Dual(x, zero(x), 1, (dfdy, parents) -> nothing)

        result = f(x)

        result.dfdy = one(result.value)
        backprop!(result)

        y = collect_outputs(result)[1]

        return y.dfdy
    end

    function dfdx(x::Array{T}, i) where T <: Number
#         print("array", "\n")
        fargs = [Dual(xelt, zero(xelt), i, (dfdy, parents) -> nothing) for (i, xelt) in enumerate(x)]
        result = f(fargs)[i]
#         println(result)
        result.dfdy = one(result.value)
        #println(result.parent)
        backprop!(result)
        y = collect_outputs(result)
        grad = zeros(typeof(result.value), length(x))
        for yelt in y
            grad[yelt.parent] = yelt.dfdy
        end

        return grad
    end

    function dfdx(x...)
        #print("x...", "\n")

        fargs = [Dual(xelt, zero(xelt), i, (dfdy, parents) -> nothing) for (i, xelt) in enumerate(x)]
        result = f(fargs...)
        result.dfdy = one(result.value)
        backprop!(result)
        y = collect_outputs(result)

        grad = zeros(typeof(result.value), length(x))
        for yelt in y
            grad[yelt.parent] = yelt.dfdy
        end

        return grad
    end

    return dfdx
end

derivativeСalculation (generic function with 1 method)

In [11]:
function derivativeСalculation(i::Integer, f)
    df = derivativeСalculation(f)
    function df_wrapper(x...)
        g = df(x...)
        return g[i]
    end
    return df_wrapper
end

derivativeСalculation (generic function with 2 methods)

In [12]:
# overloading Plus
function backpropagationForPlus!(dfdy, xy)
    x, y = xy
    x.dfdy += dfdy
    y.dfdy += dfdy
end

function +(x::Dual{T}, y::Dual{T}) where T
#     println("Przecieążenie +")
    if x == y
#         println("x = y +")
        new_y = Dual(y.value, y.dfdy, y.parent, y.backpropagation!)
        return Dual(x.value + y.value, zero(T), [x, new_y], backpropagationForPlus!)
    end
    Dual(x.value + y.value, zero(T), [x, y], backpropagationForPlus!)
end

#--------------- overloading Minus
function backpropagationForMinus!(dfdy, xy) 
    x, y = xy
    x.dfdy += dfdy
    y.dfdy -= dfdy
end

function -(x::Dual{T}, y::Dual{T}) where T
    Dual(x.value - y.value, zero(T), [x, y], backpropagationForMinus!)
end



function backpropagationForMinusOneDual!(dfdy, x)
    x.dfdy -= dfdy
end

function -(x::Dual{T}) where T
    Dual(-x.value, zero(T), x, backpropagationForMinusOneDual!)
end


#------------------- overloading Multiplication

function backpropagationForMultiplication!(dfdy, xy)
    x,y = xy
    
    x.dfdy += dfdy*y.value
    y.dfdy += x.value*dfdy
end


function *(x::Dual{T}, y::Dual{T}) where T
#     println("mnożenie", x == y)
#     println(x)
#     println(y)
    return Dual(x.value*y.value, zero(T), [x,y], backpropagationForMultiplication!)
end


#-------------------- overloading Dividing

function /(x::Dual{T}, y::Dual{T}) where T
    yinv = one(T)/y.value

    function backpropagationForDividing!(dfdy, xy)
        a,b = xy
        a.dfdy += dfdy*yinv
        b.dfdy -= a.value*dfdy*(yinv*yinv)
    end
    if x == y
#         println("x = y /")
        new_y = Dual(y.value, y.dfdy, y.parent, y.backpropagation!)
        return Dual(x.value*yinv, zero(T), [x, new_y], backpropagationForDividing!)
    end
    Dual(x.value*yinv, zero(T), [x,y], backpropagationForDividing!)
end

#-------------------- overloading EXP

function exp(x::Dual{T}) where T
    expValue = exp(x.value)

    function bp!(dfdy, p)
        p.dfdy += dfdy*expValue
    end

    Dual(expValue, zero(expValue), x, bp!)
end

function exp(xs::Array)
    return [exp(x) for x in xs]
end


#-------------------- overloading SIN
function sin(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy += cos(x.value)*dfdy
    end

    sx = sin(x.value)
    Dual(sx, zero(sx), x, bp!)
end

#-------------------- overloading COS
function cos(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy -= sin(x.value)*dfdy
    end

    cx = cos(x.value)
    Dual(cx, zero(cx), x, bp!)
end
#-------------------- overloading TAN
function tan(x::Dual{T}) where T
    c = cos(x.value)
    
    function bp!(dfdy, p)
        p.dfdy += dfdy/(c*c)
    end

    tx = tan(x.value)
    Dual(tx, zero(tx), x, bp!)
end

#---------------------- overloading SQRT
function sqrt(x::Dual{T}) where T
    sqrtValue = sqrt(x.value)

    function bp!(dfdy, p)
        p.dfdy += dfdy/(2*sqrtValue)
    end

    Dual(sqrtValue, zero(sqrtValue), x, bp!)
end

#------------------ overloading ^
function ^(a::Dual{T}, x::Dual{T}) where T
    value = a.value^x.value
#     println("potegowanie daszek")
    function bp!(dfdy, xy)
        a,x = xy
#         print(typeof(dfdy * (x.value) * (a.value)^(x.value - 1)), " nic  ", typeof(dfdy * a.value^x.value*log(a.value)))
        a.dfdy += dfdy * (x.value) * (a.value)^(x.value - 1)
        x.dfdy += dfdy * a.value^x.value*log(a.value)
    end

    Dual(value, zero(value), [a, x], bp!)
end

function ^(x::Dual{T}, n::Integer) where T
    value = x.value^n
#     println("potegowanie daszek22")
    function bp!(dfdy, x)
        x.dfdy += n * dfdy*x.value^(n-1)
    end
    
    Dual(value, zero(value), x, bp!)
end

#------------------ overloading log
function log(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy += dfdy/x.value
    end
    
    Dual(log(x.value), zero(T), x, bp!)
end
#----------------------- overloading max
function max(a, x::Dual{T}) where T
    
    function bp!(dfdy,p)
        p.dfdy += x.value < a ? a : 1 * dfdy
    end
    
    Dual(max(0, x.value), zero(T), x, bp!)
    
end


isless(x::Dual, y::Dual) = x.value < y.value;

In [13]:

# function iterate(iter::Dual, state=1)
# #     print("iterate")
#     if state > length(iter.x)
#         return nothing
#     end
#     return (iter[state],state+1)
# end

function softmax(vector::Array)
    e = exp(vector)
#     e = [MathConstants.e^(x) for x in vector]
#     s = zero(vector[1])
#     z = [Dual(2.0, 1.0, nothing, (dfdy, parents) -> nothing), Dual(21.0, 1.0, nothing, (dfdy, parents) -> nothing)]
#     for x in z
#        s = s + x 
#     end    
#     println(s, sum(e))
    return e / sum(e)
end

softmax (generic function with 1 method)

In [14]:
J = function jacobian(f, number_of_functions, args::Vector{T}) where {T <:Number}
    jacobian_rows = Matrix{T}[]
    d = derivativeСalculation(f)  
    for i=1:number_of_functions
#         x = Dual{T}[]
#         for j=1:length(args)
#             seed = (i == j)
#             push!(x, seed ?
#                 Dual(args[j], one(args[j])) :
#                 Dual(args[j],zero(args[j])) )
#         end
#         temp  = [f(x)..]
        rows = d(args,i)
        push!(jacobian_rows, rows[:,:])
    end
    jacobian_rows
end

jacobian (generic function with 1 method)

## Definicja funkcji testowych

In [15]:
function styblinskiTang(x)
    value = zero(x[1])
    for i=1:length(x)
        value += x[i]^4 - 16 * x[i]^2 + 5 * x[i]
    end
    value / 2
end

function rosenbrock(x)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end

function KarolikJeleniewiczFunction(x)
    f1 = 222.83*(x[2]^8+x[3]^-3)^2 + sin(x[5]^18) - cos(x[1])/exp(x[3])
    f2 = x[1] - 5/x[2] + 4*x[3]^2-2*x[4] / (x[3]*sin(x[5])^2 + 1)
    f3 = sqrt(exp(x[2])^2) + x[3]^x[2] - x[4]
    [f1,f2,f3]
end

function viennet(x)
    f1 = 0.5*(x[1]^2+x[2]^2) + sin(x[1]^2+ x[2]^2)
    f2 = (x[1]-2*x[2]+4)^2/8 + (x[1]-x[2]+1)^2/27 + 15
    f3 = 1/(x[1]^2+x[2]^2+1) - 1.1* exp(-(x[1]^2+x[2]^2))
    [f1,f2,f3]
end

viennet (generic function with 1 method)

## Testy funkcji weluzmiennych 

### Testowanie funkcji Rosenbrock

In [16]:
@benchmark J(rosenbrock,1,[rand(5,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  7.98 KiB
  allocs estimate:  190
  --------------
  minimum time:     6.727 μs (0.00% GC)
  median time:      7.052 μs (0.00% GC)
  mean time:        7.770 μs (7.63% GC)
  maximum time:     691.288 μs (98.18% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [17]:
@benchmark J(rosenbrock,1,[rand(50,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  76.64 KiB
  allocs estimate:  2041
  --------------
  minimum time:     76.462 μs (0.00% GC)
  median time:      79.395 μs (0.00% GC)
  mean time:        85.967 μs (6.05% GC)
  maximum time:     4.162 ms (96.24% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [18]:
@benchmark J(rosenbrock,1,[rand(500,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  741.47 KiB
  allocs estimate:  20494
  --------------
  minimum time:     748.599 μs (0.00% GC)
  median time:      784.370 μs (0.00% GC)
  mean time:        844.777 μs (5.87% GC)
  maximum time:     4.626 ms (80.05% GC)
  --------------
  samples:          5900
  evals/sample:     1

In [19]:
@benchmark J(rosenbrock,1,[rand(5000,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  7.26 MiB
  allocs estimate:  209492
  --------------
  minimum time:     8.061 ms (0.00% GC)
  median time:      8.425 ms (0.00% GC)
  mean time:        9.079 ms (7.50% GC)
  maximum time:     14.099 ms (40.23% GC)
  --------------
  samples:          551
  evals/sample:     1

In [20]:
@benchmark J(rosenbrock,1,[rand(10000,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  14.53 MiB
  allocs estimate:  419493
  --------------
  minimum time:     16.627 ms (0.00% GC)
  median time:      18.108 ms (0.00% GC)
  mean time:        19.527 ms (9.50% GC)
  maximum time:     28.943 ms (27.84% GC)
  --------------
  samples:          256
  evals/sample:     1

### Testowanie funkcji styblinskiTang

In [21]:
@benchmark J(styblinskiTang,1,[rand(5,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  10.14 KiB
  allocs estimate:  218
  --------------
  minimum time:     8.104 μs (0.00% GC)
  median time:      8.413 μs (0.00% GC)
  mean time:        9.270 μs (6.64% GC)
  maximum time:     1.161 ms (98.87% GC)
  --------------
  samples:          10000
  evals/sample:     3

In [22]:
@benchmark J(styblinskiTang,1,[rand(50,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  73.52 KiB
  allocs estimate:  1887
  --------------
  minimum time:     70.681 μs (0.00% GC)
  median time:      73.063 μs (0.00% GC)
  mean time:        79.222 μs (5.94% GC)
  maximum time:     3.266 ms (96.62% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [23]:
@benchmark J(styblinskiTang,1,[rand(500,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  696.16 KiB
  allocs estimate:  18540
  --------------
  minimum time:     708.201 μs (0.00% GC)
  median time:      725.136 μs (0.00% GC)
  mean time:        784.001 μs (6.08% GC)
  maximum time:     4.326 ms (78.72% GC)
  --------------
  samples:          6357
  evals/sample:     1

In [24]:
@benchmark J(styblinskiTang,1,[rand(5000,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  6.81 MiB
  allocs estimate:  189538
  --------------
  minimum time:     7.489 ms (0.00% GC)
  median time:      8.181 ms (0.00% GC)
  mean time:        8.978 ms (7.67% GC)
  maximum time:     16.071 ms (39.72% GC)
  --------------
  samples:          557
  evals/sample:     1

In [25]:
@benchmark J(styblinskiTang,1,[rand(10000,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  13.62 MiB
  allocs estimate:  379539
  --------------
  minimum time:     15.689 ms (0.00% GC)
  median time:      17.145 ms (0.00% GC)
  mean time:        18.401 ms (9.74% GC)
  maximum time:     26.703 ms (34.53% GC)
  --------------
  samples:          272
  evals/sample:     1

## Testy funkcji wektorowych

In [26]:
@benchmark J(viennet,2, [rand(2,1)...])

BenchmarkTools.Trial: 
  memory estimate:  11.30 KiB
  allocs estimate:  243
  --------------
  minimum time:     6.347 μs (0.00% GC)
  median time:      6.678 μs (0.00% GC)
  mean time:        7.607 μs (10.19% GC)
  maximum time:     720.508 μs (98.16% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [28]:
@benchmark J(KarolikJeleniewiczFunction,3, [rand(5,1)...])

BenchmarkTools.Trial: 
  memory estimate:  18.30 KiB
  allocs estimate:  429
  --------------
  minimum time:     16.850 μs (0.00% GC)
  median time:      17.867 μs (0.00% GC)
  mean time:        19.585 μs (6.73% GC)
  maximum time:     3.396 ms (98.93% GC)
  --------------
  samples:          10000
  evals/sample:     1

# Testy 

In [15]:
function test(a, x)
    return a*x
end

test (generic function with 1 method)

In [16]:
d = derivativeСalculation(test)

(::var"#dfdx#12"{typeof(test)}) (generic function with 3 methods)

In [17]:
d(2, 3)

x...


2-element Vector{Int64}:
 3
 2

In [18]:
# f(x::Vector) = [2x[1]*x[2], 3x[2]*x[3]^2]
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]


f (generic function with 1 method)

In [19]:
# f(x::Vector) = x[1]*x[2]*x[3]
d = derivativeСalculation(f)
d([1.0,2.0,3.0], 5)

Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"())], backpropagationForPlus!), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())], backpropagationForPlus!)]


3-element Vector{Float64}:
  0.37753914284153406
 -0.0755078285683068
 -0.0755078285683068

In [20]:
J(f,5,[1.,2., 3.]) 


1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(16.0, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(4.0, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(3.0, 0.0, 3, var"#9#15"()), Dual{Float64}(0.8414709848078965, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(1.0, 0.0, 1, var"#9#15"())))]
Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"()

5-element Vector{Matrix{Float64}}:
 [1.0; 0.0; 0.0]
 [0.0; 0.0; -0.5555555555555556]
 [0.0; 16.0; -2.0]
 [1.6209069176044193; 0.0; 0.8414709848078965]
 [0.37753914284153406; -0.0755078285683068; -0.0755078285683068]

In [21]:
J(softmax,2,[1., 2.]) 

Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(10.107337927389695, 0.0, Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(7.38905609893065, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#20"{Float64}(7.38905609893065))], backpropagationForPlus!)]
Dual{Float64}[Dual{Float64}(7.38905609893065, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#20"{Float64}(7.38905609893065)), Dual{Float64}(10.107337927389695, 0.0, Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(7.38905609893065, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#20"{Float64}(7.38905609893065))], backpropagationForPlus!)]


2-element Vector{Matrix{Float64}}:
 [0.46555335461147695; -0.19661193324148185]
 [-0.19661193324148188; 0.9276705118714866]

In [22]:
x = [Dual(1.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing), Dual(3.0, 0.0, nothing, nothing)]
y = [Dual(2.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing)]


3-element Vector{Dual{Float64}}:
 Dual{Float64}(2.0, 0.0, nothing, nothing)
 Dual{Float64}(2.0, 0.0, nothing, nothing)
 Dual{Float64}(2.0, 0.0, nothing, nothing)

In [23]:
sum(x)


Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, nothing), Dual{Float64}(2.0, 0.0, nothing, nothing)], backpropagationForPlus!), Dual{Float64}(3.0, 0.0, nothing, nothing)], backpropagationForPlus!)

In [24]:
sa(x) = sin.(x.*x)


sa (generic function with 1 method)

In [25]:
sa([2, 3])

2-element Vector{Float64}:
 -0.7568024953079282
  0.4121184852417566

In [26]:
#J(s,3, [1., 2., 3.]) 
d = derivativeСalculation(sa)
J(sa,3, [1., 2., 3.]) 

Dual{Float64}(1.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(1.0, 0.0, 1, var"#9#15"())], backpropagationForMultiplication!)
Dual{Float64}(4.0, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"())], backpropagationForMultiplication!)
Dual{Float64}(9.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, 3, var"#9#15"()), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)


3-element Vector{Matrix{Float64}}:
 [1.0806046117362795; 0.0; 0.0]
 [0.0; -2.6145744834544478; 0.0]
 [0.0; 0.0; -5.466781571308061]

In [27]:
softmax([1,2])


2-element Vector{Float64}:
 0.2689414213699951
 0.7310585786300049

In [28]:
function rosenbrock(x)
    value = zero(x[1])
#     value = (1-x[1])^2 + 100*(x[2] - x[1]^2)^2
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end

rosenbrock (generic function with 1 method)

In [29]:
rosenbrock([1.0,2.0])

100.0

In [30]:
d = D(rosenbrock)

LoadError: UndefVarError: D not defined

In [31]:
@benchmark J(rosenbrock,1,[rand(1000,1)...]) 

Dual{Float64}[Dual{Float64}(20361.97138244841, 0.0, Dual{Float64}[Dual{Float64}(20351.367535641464, 0.0, Dual{Float64}[Dual{Float64}(20351.339107716758, 0.0, Dual{Float64}[Dual{Float64}(20326.37864389372, 0.0, Dual{Float64}[Dual{Float64}(20326.152200894285, 0.0, Dual{Float64}[Dual{Float64}(20297.395712337526, 0.0, Dual{Float64}[Dual{Float64}(20297.245218057662, 0.0, Dual{Float64}[Dual{Float64}(20280.176975210103, 0.0, Dual{Float64}[Dual{Float64}(20280.113502734046, 0.0, Dual{Float64}[Dual{Float64}(20274.36735199438, 0.0, Dual{Float64}[Dual{Float64}(20197.598074127975, 0.0, Dual{Float64}[Dual{Float64}(20192.8701352898, 0.0, Dual{Float64}[Dual{Float64}(20178.30148366473, 0.0, Dual{Float64}[Dual{Float64}(20092.355187786216, 0.0, Dual{Float64}[Dual{Float64}(20010.671317887267, 0.0, Dual{Float64}[Dual{Float64}(20005.347527993126, 0.0, Dual{Float64}[Dual{Float64}(19996.729372349695, 0.0, Dual{Float64}[Dual{Float64}(19996.210359216264, 0.0, Dual{Float64}[Dual{Float64}(19988.519377617962, 0.0,

(16372.67476322351, 0.0, Dual{Float64}[Dual{Float64}(16343.193346979211, 0.0, Dual{Float64}[Dual{Float64}(16294.07167745528, 0.0, Dual{Float64}[Dual{Float64}(16212.056370877652, 0.0, Dual{Float64}[Dual{Float64}(16173.188126099752, 0.0, Dual{Float64}[Dual{Float64}(16172.86411526021, 0.0, Dual{Float64}[Dual{Float64}(16171.651493791978, 0.0, Dual{Float64}[Dual{Float64}(16171.55618549195, 0.0, Dual{Float64}[Dual{Float64}(16086.155283941658, 0.0, Dual{Float64}[Dual{Float64}(16084.602574648388, 0.0, Dual{Float64}[Dual{Float64}(16068.600941177432, 0.0, Dual{Float64}[Dual{Float64}(16067.792619537055, 0.0, Dual{Float64}[Dual{Float64}(15981.64900305067, 0.0, Dual{Float64}[Dual{Float64}(15921.628375182356, 0.0, Dual{Float64}[Dual{Float64}(15832.24357107558, 0.0, Dual{Float64}[Dual{Float64}(15819.563437609673, 0.0, Dual{Float64}[Dual{Float64}(15769.047148161106, 0.0, Dual{Float64}[Dual{Float64}(15709.989200025835, 0.0, Dual{Float64}[Dual{Float64}(15634.486324543348, 0.0, Dual{Float64}[Dual{Float64

, 0.0, Dual{Float64}[Dual{Float64}(12303.962005305886, 0.0, Dual{Float64}[Dual{Float64}(12228.612326801593, 0.0, Dual{Float64}[Dual{Float64}(12165.451439022465, 0.0, Dual{Float64}[Dual{Float64}(12163.002775356983, 0.0, Dual{Float64}[Dual{Float64}(12138.415795906654, 0.0, Dual{Float64}[Dual{Float64}(12111.121881058092, 0.0, Dual{Float64}[Dual{Float64}(12056.873491569964, 0.0, Dual{Float64}[Dual{Float64}(12056.606486048651, 0.0, Dual{Float64}[Dual{Float64}(12050.457278545897, 0.0, Dual{Float64}[Dual{Float64}(12004.687268699152, 0.0, Dual{Float64}[Dual{Float64}(12002.059211824902, 0.0, Dual{Float64}[Dual{Float64}(11995.857785970578, 0.0, Dual{Float64}[Dual{Float64}(11988.03535103378, 0.0, Dual{Float64}[Dual{Float64}(11978.820009111727, 0.0, Dual{Float64}[Dual{Float64}(11969.969389105865, 0.0, Dual{Float64}[Dual{Float64}(11967.404341991838, 0.0, Dual{Float64}[Dual{Float64}(11939.462659515626, 0.0, Dual{Float64}[Dual{Float64}(11925.479910289536, 0.0, Dual{Float64}[Dual{Float64}(11874.729948

Dual{Float64}(8440.057153411657, 0.0, Dual{Float64}[Dual{Float64}(8430.161718165427, 0.0, Dual{Float64}[Dual{Float64}(8401.82839865242, 0.0, Dual{Float64}[Dual{Float64}(8384.525755750017, 0.0, Dual{Float64}[Dual{Float64}(8384.477452465513, 0.0, Dual{Float64}[Dual{Float64}(8384.326341263111, 0.0, Dual{Float64}[Dual{Float64}(8330.42568148513, 0.0, Dual{Float64}[Dual{Float64}(8314.653721177023, 0.0, Dual{Float64}[Dual{Float64}(8303.589091781412, 0.0, Dual{Float64}[Dual{Float64}(8293.166792885455, 0.0, Dual{Float64}[Dual{Float64}(8293.11420671189, 0.0, Dual{Float64}[Dual{Float64}(8234.057682552058, 0.0, Dual{Float64}[Dual{Float64}(8210.937899344568, 0.0, Dual{Float64}[Dual{Float64}(8153.221330955286, 0.0, Dual{Float64}[Dual{Float64}(8152.518745235099, 0.0, Dual{Float64}[Dual{Float64}(8151.416711317877, 0.0, Dual{Float64}[Dual{Float64}(8122.846529344923, 0.0, Dual{Float64}[Dual{Float64}(8093.46040126632, 0.0, Dual{Float64}[Dual{Float64}(8091.750239244925, 0.0, Dual{Float64}[Dual{Float64}(80

, 0.0, Dual{Float64}[Dual{Float64}(3187.915005092099, 0.0, Dual{Float64}[Dual{Float64}(3165.687063039488, 0.0, Dual{Float64}[Dual{Float64}(3151.500778605787, 0.0, Dual{Float64}[Dual{Float64}(3149.3209380282983, 0.0, Dual{Float64}[Dual{Float64}(3078.60181338559, 0.0, Dual{Float64}[Dual{Float64}(2992.0927585702448, 0.0, Dual{Float64}[Dual{Float64}(2991.2579239581178, 0.0, Dual{Float64}[Dual{Float64}(2982.70483946022, 0.0, Dual{Float64}[Dual{Float64}(2938.0972207851464, 0.0, Dual{Float64}[Dual{Float64}(2936.864510747419, 0.0, Dual{Float64}[Dual{Float64}(2929.869708061862, 0.0, Dual{Float64}[Dual{Float64}(2921.259860129203, 0.0, Dual{Float64}[Dual{Float64}(2921.1775621884926, 0.0, Dual{Float64}[Dual{Float64}(2902.8857890154113, 0.0, Dual{Float64}[Dual{Float64}(2892.670019254983, 0.0, Dual{Float64}[Dual{Float64}(2877.031710362193, 0.0, Dual{Float64}[Dual{Float64}(2870.251803829468, 0.0, Dual{Float64}[Dual{Float64}(2867.5250216990325, 0.0, Dual{Float64}[Dual{Float64}(2847.8851450611437, 0.0,

...

[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.33871593244627446, 0.0, Dual{Float64}(0.5819930690706501, 0.0, Dual{Float64}[Dual{Float64}(0.589118797737505, 0.0, 535, var"#9#15"()), Dual{Float64}(0.007125728666854867, 0.0, Dual{Float64}(0.08441403122025903, 0.0, 534, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMinus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(0.3360467436690745, 0.0, Dual{Float64}[Dual{Float64}(0.16882336237267334, 0.0, Dual{Float64}(0.410881202262495, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.589118797737505, 0.0, 535, var"#9#15"())], backpropagationForMinus!), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.16722338129640116, 0.0, Dual{Float64}[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.0016722338129640116, 0.0, Dual{Float64}(0.04089295554204919, 0.0, Dual{Float64}[Dual{Float64}(0.38795391338

(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.7746874137112629, 0.0, 552, var"#9#15"())], backpropagationForMinus!), var"#bp!#26"{Int64}(2)), Dual{Float64}(10.720382547629624, 0.0, Dual{Float64}[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.10720382547629624, 0.0, Dual{Float64}(-0.32741995277669966, 0.0, Dual{Float64}[Dual{Float64}(0.2727206361859458, 0.0, 553, var"#9#15"()), Dual{Float64}(0.6001405889626454, 0.0, Dual{Float64}(0.7746874137112629, 0.0, 552, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMinus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(1.4556670828586218, 0.0, Dual{Float64}[Dual{Float64}(0.5289352730297755, 0.0, Dual{Float64}(0.7272793638140542, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.2727206361859458, 0.0, 553, var"#9#15"())], backpropagationForMinus!), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.92673180982

Excessive output truncated after 524290 bytes.

, 0.0, Dual{Float64}[Dual{Float64}(0.16896422452306203, 0.0, 565, var"#9#15"()), Dual{Float64}(0.4140267326699629, 0.0, Dual{Float64}(0.6434490909698785, 0.0, 564, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMinus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(16.959562903825038, 0.0, Dual{Float64}[Dual{Float64}(0.6906204601225556, 0.0, Dual{Float64}(0.831035775476938, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.16896422452306203, 0.0, 565, var"#9#15"())], backpropagationForMinus!), var"#bp!#26"{Int64}(2)), Dual{Float64}(16.268942443702482, 0.0, Dual{Float64}[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.16268942443702483, 0.0, Dual{Float64}(0.40334777108225706, 0.0, Dual{Float64}[Dual{Float64}(0.43189668025093675, 0.0, 566, var"#9#15"()), Dual{Float64}(0.028548909168679717, 0.0, Dual{Float64}(0.16896422452306203, 0.0, 565, var"#9

BenchmarkTools.Trial: 
  memory estimate:  122.18 MiB
  allocs estimate:  3985565
  --------------
  minimum time:     3.501 s (0.20% GC)
  median time:      3.582 s (0.25% GC)
  mean time:        3.582 s (0.25% GC)
  maximum time:     3.663 s (0.31% GC)
  --------------
  samples:          2
  evals/sample:     1

In [32]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
@benchmark J(f,6,[rand(3,1)...]) 

1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.9278147270301493, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.44197090209088147, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.11049272552272037, 0.0, Dual{Float64}(0.3324044607443173, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.8556294540602987, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.9278147270301493, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(0.9278147270301493, 0.0, 3, var"#9#15"()), Dual{Float64}(0.809329146570272, 0.0, Dual{Float64}(0.9430090553690504, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(0.9430090553690504, 0.0, 1, var"#9#15"())))]
Dual{Float64}[Dual{Float64}(2.567696145360134, 0.0, Dual{Float64}(0.9430090553690504, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.567696145360134)), Dual{Fl

Dual{Float64}[Dual{Float64}(-0.13660786173867603, 0.0, Dual{Float64}[Dual{Float64}(0.02694260781879742, 0.0, Dual{Float64}(0.16414203550217543, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.16355046955747346, 0.0, Dual{Float64}(0.5468697886054166, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.38264221312726643, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.6671153416814858, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.16677883542037145, 0.0, Dual{Float64}(0.4083856454631718, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.7652844262545329, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.38264221312726643, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Fl

Dual{Float64}[Dual{Float64}(0.5561266044382811, 0.0, Dual{Float64}[Dual{Float64}(0.7000253142481297, 0.0, Dual{Float64}(0.8366751545541016, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.1438987098098487, 0.0, Dual{Float64}(0.5240253539995132, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.1277199590386966, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.06862240447834056, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.01715560111958514, 0.0, Dual{Float64}(0.13097939196524444, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.2554399180773932, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.1277199590386966, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64

Dual{Float64}[Dual{Float64}(-0.5314185944721148, 0.0, Dual{Float64}[Dual{Float64}(0.2359539586387636, 0.0, Dual{Float64}(0.4857509224270846, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.7673725531108784, 0.0, Dual{Float64}(0.9155219346401944, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.5029616133023656, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.5985161718819292, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.1496290429704823, 0.0, Dual{Float64}(0.38681913470055007, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.005923226604731, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.5029616133023656, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(

Dual{Float64}[Dual{Float64}(-0.2868759943174616, 0.0, Dual{Float64}[Dual{Float64}(0.04551253653791487, 0.0, Dual{Float64}(0.2133366741512459, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.33238853085537645, 0.0, Dual{Float64}(0.6927055649939164, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.9312578770950253, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(3.6480043069587813, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.9120010767396953, 0.0, Dual{Float64}(0.9549874746506863, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.8625157541900506, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.9312578770950253, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64

...

Dual{Float64}[Dual{Float64}(-0.571643676992438, 0.0, Dual{Float64}[Dual{Float64}(0.007230171230517943, 0.0, Dual{Float64}(0.08503041356195995, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.578873848222956, 0.0, Dual{Float64}(0.8334149947000185, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.7364044724047623, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(2.9048597006690495, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.7262149251672624, 0.0, Dual{Float64}(0.8521824482863176, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.4728089448095245, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.7364044724047623, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}

Dual{Float64}[Dual{Float64}(-0.21827939940357222, 0.0, Dual{Float64}[Dual{Float64}(0.025117934808761306, 0.0, Dual{Float64}(0.1584863868247406, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.24339733421233353, 0.0, Dual{Float64}(0.6243650804163314, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.1473342331402303, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(2.0042336764176336, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.5010584191044084, 0.0, Dual{Float64}(0.7078548008627252, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.2946684662804606, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.1473342331402303, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float

Excessive output truncated after 524305 bytes.

{Float64}(-0.3521365792044213, 0.0, Dual{Float64}[Dual{Float64}(0.2422504524257473, 0.0, Dual{Float64}(0.4921894476984927, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.5943870316301686, 0.0, Dual{Float64}(0.8407943315868514, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.17897319422860103, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.7598080941632821, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.18995202354082052, 0.0, Dual{Float64}(0.43583485810662337, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.35794638845720206, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.17897319422860103, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(0.17897319422

BenchmarkTools.Trial: 
  memory estimate:  276.67 KiB
  allocs estimate:  8684
  --------------
  minimum time:     1.972 ms (0.00% GC)
  median time:      2.045 ms (0.00% GC)
  mean time:        2.094 ms (0.86% GC)
  maximum time:     5.872 ms (59.78% GC)
  --------------
  samples:          2383
  evals/sample:     1

In [33]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
J(f,6,[2.9,2.2,1546.23]) 


1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(1546.23, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(19.360000000000003, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(3092.46, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(1546.23, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(1546.23, 0.0, 3, var"#9#15"()), Dual{Float64}(0.23924932921398243, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(2.9, 0.0, 1, var"#9#15"())))]
Dual{Float64}[Dual{Float64}(18.17414536944306, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(18.17414536944306)), Dual{Float64}(1551.33, 0.0, Dual{Float64}[Dual{Float64}(5.1, 0.0, Dual{Float64}[Dual{Float64}(2.9, 0.0, 1, var"#

6-element Vector{Matrix{Float64}}:
 [1.0; 0.0; 0.0]
 [0.0; 0.0; -2.0913263714842668e-6]
 [0.0; 17.6; -2.0]
 [-1501.3246436992515; 0.0; 0.23924932921398243]
 [0.011707650961926682; -7.55171541667044e-6; -7.55171541667044e-6]
 [5.8; -14.520000000000003; 0.0]

In [34]:
function s(vector)
    vector[1] * vector[2] + sin(vector[1])
end

s (generic function with 1 method)

In [35]:
d = derivativeСalculation(s)

(::var"#dfdx#12"{typeof(s)}) (generic function with 3 methods)

In [36]:
d([1.0, 2.0], 1)

Dual{Float64}[Dual{Float64}(2.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"())], backpropagationForMultiplication!), Dual{Float64}(0.8414709848078965, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(1.0, 0.0, 1, var"#9#15"())))]


2-element Vector{Float64}:
 2.5403023058681398
 1.0

In [38]:
d = derivativeСalculation(styblinskiTang)

(::var"#dfdx#12"{typeof(styblinskiTang)}) (generic function with 3 methods)

In [39]:
d([2.9,2.2],1)

Dual{Float64}[Dual{Float64}(-92.34630000000001, 0.0, Dual{Float64}[Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(0.0, 0.0, nothing, var"#5#6"()), Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(-63.831900000000005, 0.0, Dual{Float64}[Dual{Float64}(70.7281, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(4)), Dual{Float64}(134.56, 0.0, Dual{Float64}[Dual{Float64}(16.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForMinus!), Dual{Float64}(14.5, 0.0, Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(2.9, 0.0, 1, var"#9#15"())], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(-43.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(-54.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(23.425600000000006, 0.0, Dual{Float64}(2.2, 

2-element Vector{Float64}:
   4.878
 -11.403999999999996

In [40]:
J(styblinskiTang, 1,[2.9, 2.2])

Dual{Float64}[Dual{Float64}(-92.34630000000001, 0.0, Dual{Float64}[Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(0.0, 0.0, nothing, var"#5#6"()), Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(-63.831900000000005, 0.0, Dual{Float64}[Dual{Float64}(70.7281, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(4)), Dual{Float64}(134.56, 0.0, Dual{Float64}[Dual{Float64}(16.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForMinus!), Dual{Float64}(14.5, 0.0, Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(2.9, 0.0, 1, var"#9#15"())], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(-43.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(-54.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(23.425600000000006, 0.0, Dual{Float64}(2.2, 

1-element Vector{Matrix{Float64}}:
 [4.878; -11.403999999999996]

In [41]:
J(viennet, 3, [2.9,2.2])

Dual{Float64}[Dual{Float64}(6.625, 0.0, Dual{Float64}[Dual{Float64}(0.5, 0.0, nothing, var"#1#2"()), Dual{Float64}(13.25, 0.0, Dual{Float64}[Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForPlus!)], backpropagationForMultiplication!), Dual{Float64}(0.6316109877182386, 0.0, Dual{Float64}(13.25, 0.0, Dual{Float64}[Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForPlus!), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(13.25, 0.0, Dual{Float64}[Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForPlus!)))]
D

3-element Vector{Matrix{Float64}}:
 [7.39665572674987; 5.611256068568867]
 [0.7509259259259258; -1.3759259259259258]
 [-0.028551403647345923; -0.02165968552557277]

In [42]:
z = J(KarolikJeleniewiczFunction, 3, [2.9,2.2,2.2,34456.5,1.8])

Dual{Float64}[Dual{Float64}(6.712513875660308e7, 0.0, Dual{Float64}[Dual{Float64}(6.712513786463414e7, 0.0, Dual{Float64}[Dual{Float64}(222.83, 0.0, nothing, var"#1#2"()), Dual{Float64}(301239.23109381204, 0.0, Dual{Float64}(548.852649710113, 0.0, Dual{Float64}[Dual{Float64}(548.7587353600004, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(8)), Dual{Float64}(0.0939143501126972, 0.0, Dual{Float64}(0.45454545454545453, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(2.2, 0.0, 3, var"#9#15"())], var"#backpropagationForDividing!#19"{Float64}(0.45454545454545453)), var"#bp!#26"{Int64}(3))], backpropagationForPlus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.891968929252866, 0.0, Dual{Float64}(39346.40807529655, 0.0, Dual{Float64}(1.8, 0.0, 5, var"#9#15"()), var"#bp!#26"{Int64}(18)), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(39346.40807529655, 0.0, Dual{Float64}(1.8, 0.0, 5, var"#9#15"()), var"#bp!#26"{In

3-element Vector{Matrix{Float64}}:
 [0.026509581312979046; 4.880992877171569e8; … ; 0.0; 177883.81382445205]
 [1.0; 1.0330578512396693; … ; -0.6479969546855062; -7042.763658371653]
 [0.0; 13.492961495134765; … ; -1.0; 0.0]

In [43]:
println(z)

[[0.026509581312979046; 4.880992877171569e8; -31325.026289957223; 0.0; 177883.81382445205], [1.0; 1.0330578512396693; 6878.310898259487; -0.6479969546855062; -7042.763658371653], [0.0; 13.492961495134765; 5.666695778750081; -1.0; 0.0]]
